## Script de extração de dados do JusBrasil

Os Códigos abaixo contém o passo a passo feito para conseguir realizar com sucesso a extração de dados da planilha fornecida pela AECOM com os links dos processos

In [ ]:
#bibliotecas

import os
import pandas as pd
import requests
from bs4 import BeautifulSoup
import tempfile
import pymupdf
import re
from dotenv import load_dotenv
import google.generativeai as genai
import time

In [ ]:
# excel fornecido
anexos = pd.read_excel('docs/jusbrasil.xlsx', sheet_name="Anexos")

In [ ]:
# filtrando os anexos para manter apenas os relevantes
anexos_copia_min = anexos.copy().loc[:,["processoID", "processoAnexoID", "Download copia", "Tipo de anexo", "Tribunal"]]
processos_filtrados = anexos_copia_min.loc[anexos["Tipo de anexo"].isin(["SENTENCA", "DECISAO", "DOCUMENTO", "LAUDO"]),
                                           ["processoID", "processoAnexoID", "Download copia", "Tribunal"]]

In [21]:
processos_filtrados

,processoID,processoAnexoID,Download copia,Tribunal
0,474474022,2344968075,http://jud-anexos.digesto.com.br/5ab34519c8f96...,TJAM
1,474474022,2344968081,http://jud-anexos.digesto.com.br/0cacd6d80c499...,TJAM
2,474474806,2294638419,http://jud-anexos.digesto.com.br/2adc94fb608d8...,TJAM
6,474474806,2294638453,http://jud-anexos.digesto.com.br/7abb2bb3a616f...,TJAM
7,474474806,2294638461,http://jud-anexos.digesto.com.br/90df1f9ac9917...,TJAM
...,...,...,...,...
14451,654842012,2345563383,http://jud-anexos.digesto.com.br/bf30bec0d4dce...,TRF6
14581,654842012,2345563494,http://jud-anexos.digesto.com.br/f74beff7c3256...,TRF6
14621,654842012,2345563552,http://jud-anexos.digesto.com.br/c61c5196d583c...,TRF6
14898,654842012,2345563802,http://jud-anexos.digesto.com.br/37bb63e088989...,TRF6


### Capturando os números dos processos por regex

In [ ]:
def captura_num_processos(filtrados):
    copia_filtrados = filtrados.copy()
    sucessos = 0
    processos = []

    for i in range(copia_filtrados.shape[0]):
        link = copia_filtrados.iloc[i, 2]
        response = requests.get(link)
        response.encoding = 'utf-8'
        content_type = response.headers.get('Content-Type', '')
        texto = ""
        num_processo = "Não encontrado"

        if 'pdf' in content_type:
            # print(f"[PDF] Extraindo de: {link}")
            # Criando arquivo temporário para ler conteúdo do PDF 
            with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
                tmp_file.write(response.content)
                tmp_path = tmp_file.name

            # Abrindo arquivo temporário e extraindo texto 
            with pymupdf.open(tmp_path) as doc:
                for page in doc:
                    texto += page.get_text()
            
        elif 'html' in content_type:
            # print(f"[HTML] Extraindo de: {link}")
            soup = BeautifulSoup(response.content, 'html.parser')
            texto = soup.get_text()

        match = re.search(r'\d{7}-\d{2}\.\d{4}\.\d\.\d{2}\.\d{4}', texto)
        if match:
            num_processo = match.group(0)
            sucessos += 1

        processos.append(num_processo)
    
    print(f"Números de processos capturados: {sucessos}")

    copia_filtrados["num_processo"] = processos
    return copia_filtrados
    
filtrados_com_num = captura_num_processos(processos_filtrados)
# filtrados_com_num.to_excel("processos_fitrados.xlsx")
filtrados_com_num

MuPDF error: format error: object is not a stream

MuPDF error: format error: object is not a stream

MuPDF error: format error: object is not a stream

MuPDF error: format error: object is not a stream

MuPDF error: format error: object is not a stream

MuPDF error: format error: object is not a stream

MuPDF error: format error: object is not a stream

MuPDF error: format error: object is not a stream

MuPDF error: format error: object is not a stream

MuPDF error: format error: object is not a stream

MuPDF error: format error: object is not a stream

MuPDF error: format error: object is not a stream

MuPDF error: format error: object is not a stream

MuPDF error: format error: object is not a stream

MuPDF error: format error: object is not a stream

MuPDF error: format error: object is not a stream

MuPDF error: format error: object is not a stream

MuPDF error: format error: object is not a stream

MuPDF error: format error: object is not a stream

MuPDF error: format error: obje

,processoID,processoAnexoID,Download copia,Tribunal,num_processo
0,474474022,2344968075,http://jud-anexos.digesto.com.br/5ab34519c8f96...,TJAM,0600357-89.2021.8.04.5600
1,474474022,2344968081,http://jud-anexos.digesto.com.br/0cacd6d80c499...,TJAM,0600357-89.2021.8.04.5600
2,474474806,2294638419,http://jud-anexos.digesto.com.br/2adc94fb608d8...,TJAM,0600384-72.2021.8.04.5600
6,474474806,2294638453,http://jud-anexos.digesto.com.br/7abb2bb3a616f...,TJAM,0600384-72.2021.8.04.5600
7,474474806,2294638461,http://jud-anexos.digesto.com.br/90df1f9ac9917...,TJAM,0600384-72.2021.8.04.5600
...,...,...,...,...,...
14451,654842012,2345563383,http://jud-anexos.digesto.com.br/bf30bec0d4dce...,TRF6,1000415-46.2020.4.01.3800
14581,654842012,2345563494,http://jud-anexos.digesto.com.br/f74beff7c3256...,TRF6,1000415-46.2020.4.01.3800
14621,654842012,2345563552,http://jud-anexos.digesto.com.br/c61c5196d583c...,TRF6,1000415-46.2020.4.01.3800
14898,654842012,2345563802,http://jud-anexos.digesto.com.br/37bb63e088989...,TRF6,1000415-46.2020.4.01.3800


In [ ]:
print(filtrados_com_num.shape)

(4298, 5)


In [24]:
filtrados_com_num.loc[filtrados_com_num['num_processo'] != "Não encontrado", :]['num_processo'].unique()

array(['0600357-89.2021.8.04.5600', '0600384-72.2021.8.04.5600',
       '0681058-19.2022.8.04.0001', '0756053-37.2021.8.04.0001',
       '0471756-76.2024.8.04.0001', '0097690-14.2024.8.04.1000',
       '0600592-22.2022.8.04.5600', '8001732-18.2024.8.05.0261',
       '5083304-83.2019.8.09.0011', '5284867-26.2022.8.09.0011',
       '5580808-19.2022.8.09.0011', '5061164-56.2024.8.09.0051',
       '5589610-60.2024.8.09.0132', '5983628-49.2024.8.09.0051',
       '5921614-29.2024.8.09.0051', '1000009-75.2018.4.01.3903',
       '1000305-06.2018.4.01.3901', '1003105-10.2017.4.01.3200',
       '5096053-69.2018.8.09.0011', '5125255-42.2023.8.09.0100',
       '5456795-74.2023.8.09.0087', '5687936-95.2023.8.09.0130',
       '5842003-89.2023.8.09.0074', '0000156-20.2023.8.02.0076',
       '0700947-45.2023.8.02.0076', '0700160-79.2024.8.02.0076',
       '0700066-27.2024.8.02.0143', '0700580-84.2024.8.02.0076',
       '0700971-67.2023.8.02.0078', '0002171-93.2018.8.08.0035',
       '0233366-06.2020.8

In [25]:
processos_nao_encontrados = filtrados_com_num.loc[filtrados_com_num['num_processo'] == "Não encontrado", :]

##  Acesso negado
Alguns documentos do jusbrasil não possuiam possuiam acesso livre.

In [ ]:
def identifica_acesso_negado(processos):
    copia_processos = processos.copy()
    acessos_negados = []

    for i in range(copia_processos.shape[0]):
        link = copia_processos.iloc[i, 2]
        response = requests.get(link)
        response.encoding = 'utf-8'
        content_type = response.headers.get('Content-Type', '')
        texto = ""

        if 'html' in content_type:
            # print(f"[HTML] Extraindo de: {link}")
            soup = BeautifulSoup(response.content, 'html.parser')
            texto = soup.get_text()

        match = re.search(r'Acesso negado', texto) #identificação do acesso negado
        if match:
            acessos_negados.append(i)
            continue
    
    return copia_processos.iloc[acessos_negados, :]

df_html_only = processos_nao_encontrados[processos_nao_encontrados['Download copia'].str.contains(r'\.html?$', case=False, na=False)]
acessos_negados = identifica_acesso_negado(df_html_only)

In [32]:
acessos_negados

,processoID,processoAnexoID,Download copia,Tribunal,num_processo
1293,471515190,727395468,http://jud-anexos.digesto.com.br/3a9fbbe2c4f69...,TJMG,Não encontrado
1302,471515190,727395467,http://jud-anexos.digesto.com.br/7767cbbc02e35...,TJMG,Não encontrado
2738,583194759,2715510017,http://jud-anexos.digesto.com.br/994c084da9bb0...,TJMG,Não encontrado
2876,595998611,1403409515,http://jud-anexos.digesto.com.br/0ec3e0c015d72...,TJMG,Não encontrado
2877,595998611,1403409516,http://jud-anexos.digesto.com.br/385306277d658...,TJMG,Não encontrado
2896,595998611,1403409514,http://jud-anexos.digesto.com.br/c02f589d0ced5...,TJMG,Não encontrado
7998,690025576,2015456648,http://jud-anexos.digesto.com.br/d089d25a1198f...,TJMG,Não encontrado
8000,690025576,2015456650,http://jud-anexos.digesto.com.br/fb01c9fd839aa...,TJMG,Não encontrado


In [37]:
processos_nao_encontrados["Download copia"].unique()

array(['http://jud-anexos.digesto.com.br/30f8d4e93b94a25a3ec1b119ba506b42.pdf',
       'http://jud-anexos.digesto.com.br/f2d578e94d700989b63b06e665c63716.pdf',
       'http://jud-anexos.digesto.com.br/ac2380dd6d9520fe95ea49941097d892.pdf',
       'http://jud-anexos.digesto.com.br/042db8c60760608974e54a096960437b.pdf',
       'http://jud-anexos.digesto.com.br/a34562b835d2af3f5ef7648d5ee9ea25.pdf',
       'http://jud-anexos.digesto.com.br/396b9f3dbabc04696f800500c367ffb7.pdf',
       'http://jud-anexos.digesto.com.br/4e2d68486b63eadb0227339981c6b300.pdf',
       'http://jud-anexos.digesto.com.br/313133a43d0b9da3319b9839805c2cda.html',
       'http://jud-anexos.digesto.com.br/2a8bb47194987e02c543ed2f81cbc90e.html',
       'http://jud-anexos.digesto.com.br/d37811c8de5edf852bc8d226d0a374ef.html',
       'http://jud-anexos.digesto.com.br/7d84f5a58d46a2f49d6094e101a6f661.pdf',
       'http://jud-anexos.digesto.com.br/ce77a1930da0461256f830e5f7f58023.html',
       'http://jud-anexos.digesto.co

### Tentativa de filtro dos processos relevantes, georreferências, áreas afetadas e valores de multa

As informações relevantes vêm com uma determinada estrutura. Neste código existe a tentaiva de captura dessas informações levando em consideração sua estrutura.

In [ ]:
def captura_melhorada(filtrados):
    copia_filtrados = filtrados.copy()
    processos = 0
    georreferencias = 0
    extensoes = 0
    multas = 0
    processos_lista = []
    georreferencias_lista = []
    extensoes_lista = []
    multas_lista = []
    # palavras_chave_ambientais = [
    #     "impacto ambiental", "crime ambiental", "meio ambiente", "desmatamento",
    #     "poluição", "resíduo tóxico", "reparação ambiental", "contaminação"
    # ]

    for i in range(copia_filtrados.shape[0]):
        link = copia_filtrados.iloc[i, 2]
        response = requests.get(link)
        response.encoding = 'utf-8'
        content_type = response.headers.get('Content-Type', '')
        texto = ""
        process_number = "Não encontrado"
        georref = "Não encontrado"
        extension = "Não encontrado"
        fine = "Não encontrado"

        if 'pdf' in content_type:
            # print(f"[PDF] Extraindo de: {link}")
            # Criando arquivo temporário para ler conteúdo do PDF 
            with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
                tmp_file.write(response.content)
                tmp_path = tmp_file.name

            # Abrindo arquivo temporário e extraindo texto 
            with pymupdf.open(tmp_path) as doc:
                for page in doc:
                    texto += page.get_text()
            
        elif 'html' in content_type:
            # print(f"[HTML] Extraindo de: {link}")
            soup = BeautifulSoup(response.content, 'html.parser')
            texto = soup.get_text()

        # Número do processo
        match = re.search(r'\d{7}-\d{2}\.\d{4}\.\d\.\d{2}\.\d{4}', texto)
        if match:
            process_number = match.group(0)
            processos += 1

        # Georreferência
        match = re.search(r'\d{2}[º°]\d{2}[’\']\d{2}[”"]?\s*[NS],?\s*\d{2,3}[º°]\d{2}[’\']\d{2}[”"]?\s*[OLW]', texto)
        if match:
            georref = match.group(0)
            georreferencias += 1

        # Extensão afetada
        match = re.search(r'\d+(?:,\d+)?\s*(?:hectares|km²|m²|ha)', texto, re.IGNORECASE)
        if match:
            extension = match.group(0).strip()
            extensoes += 1

        # Multa (se existir)
        match = re.search(r'R\$ ?\d{1,3}(?:\.\d{3})*,\d{2}', texto)
        if match:
            fine = match.group(0).strip()
            multas += 1

        processos_lista.append(process_number)
        georreferencias_lista.append(georref)
        extensoes_lista.append(extension)
        multas_lista.append(fine)
    
    print(f"Números de processos capturados: {processos}")
    print(f"Números de georreferências capturadas: {georreferencias}")
    print(f"Números de extensões capturadas: {extensoes}")
    print(f"Números de multas capturadas: {multas}")

    copia_filtrados["num_processo"] = processos_lista
    copia_filtrados["georreferência"] = georreferencias_lista
    copia_filtrados["extensão"] = extensoes_lista
    copia_filtrados["multa"] = multas_lista
    return copia_filtrados

filtrados_v2 = captura_melhorada(processos_filtrados)
filtrados_v2

Números de processos capturados: 381
Números de georreferências capturadas: 1
Números de extensões capturadas: 47
Números de multas capturadas: 279


,processoID,processoAnexoID,Download copia,Tribunal,num_processo,georreferência,extensão,multa
1,474474022,2344968081,http://jud-anexos.digesto.com.br/0cacd6d80c499...,TJAM,0600357-89.2021.8.04.5600,Não encontrado,Não encontrado,Não encontrado
7,474474806,2294638461,http://jud-anexos.digesto.com.br/90df1f9ac9917...,TJAM,0600384-72.2021.8.04.5600,07º43’11” S 61º28’11” W,"121,15ha","R$ 1.301.393,30"
33,575718248,1511375012,http://jud-anexos.digesto.com.br/c89c7a26febcc...,TJAM,0681058-19.2022.8.04.0001,Não encontrado,Não encontrado,Não encontrado
87,580649179,1504080402,http://jud-anexos.digesto.com.br/585fdfebfdd6d...,TJAM,0471756-76.2024.8.04.0001,Não encontrado,Não encontrado,"R$8.000,00"
105,654928926,2419043700,http://jud-anexos.digesto.com.br/6be7e9beac86d...,TJAM,0097690-14.2024.8.04.1000,Não encontrado,Não encontrado,"R$ 8.971,31"
...,...,...,...,...,...,...,...,...
13643,494408691,2315018026,http://jud-anexos.digesto.com.br/8ab01c9d05320...,TRF1,1000098-88.2019.4.01.3604,Não encontrado,"422,58 hectares","R$ 10.742,00"
13876,494410810,269258456,http://jud-anexos.digesto.com.br/e55b90bdcd194...,TRF1,1002190-78.2020.4.01.3903,Não encontrado,"71,9 hectares","R$ 1.160.136,00"
13877,494410810,2273754745,http://jud-anexos.digesto.com.br/e3f977a11a4d1...,TRF1,1002190-78.2020.4.01.3903,Não encontrado,"71,9 hectares","R$ 10.000.000,00"
13903,549928831,1874259862,http://jud-anexos.digesto.com.br/4c97bd2fdb061...,TRF1,1000652-26.2020.4.01.3818,Não encontrado,Não encontrado,Não encontrado


### Nova abordagem: API de LLMs

A abordagem anterior possuia alguns problemas, pois não necessariamente a estrutura encontrada corresponde aos dados do próprio processo, ou seja, uma informação relevante. Por isso, foi implementada uma nova abordagem a fim de que o texto seja interpretado por uma LLM, sendo capaz de levar em consideração o contexto do texto e das informações presentes nele.

In [ ]:
load_dotenv()
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
genai.configure(api_key=GEMINI_API_KEY)

filtrados = pd.read_excel("docs/processos_filtrados.xlsx")
response = requests.get(filtrados.iloc[1499,3])
response.encoding = 'utf-8'
content_type = response.headers.get('Content-Type', '')
texto_extraido = ""
if 'pdf' in content_type:
    # print(f"[PDF] Extraindo de: {link}")
    # Criando arquivo temporário para ler conteúdo do PDF 
    with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
        tmp_file.write(response.content)
        tmp_path = tmp_file.name

    # Abrindo arquivo temporário e extraindo texto 
    with pymupdf.open(tmp_path) as doc:
        for page in doc:
            texto_extraido += page.get_text()
    
elif 'html' in content_type:
    # print(f"[HTML] Extraindo de: {link}")
    soup = BeautifulSoup(response.content, 'html.parser')
    texto_extraido = soup.get_text()

# Prompt
prompt = f"""Você está recebendo um texto bruto de um documento sobre
        algum processo judiciário. O seu interesse principal é descobrir se este texto trata de um caso relacionado à 
        valoração de danos ambientais.
        Assim, a partir das informações nele contidas, devolva APENAS UMA LISTA (NÃO DEVOLVA O TEXTO) 
        com as seguintes informações em ordem:
        Elemento 0 da lista: Texto fala de um processo em que houve algum tipo de dano ambiental (True ou False)
        Elemento 1 da lista: Texto menciona uma georreferência do local afetado pelo dano ambiental (devolva a string
        "Não encontrado" caso não exista)
        Elemento 2 da lista: A UF do local em que houve o dano ambiental (devolva a string
        "Não encontrado" caso não exista)
        Elemento 3 da lista: O município do local em que houve o dano ambiental (devolva a string
        "Não encontrado" caso não exista)
        Elemento 4 da lista: O responsável pelo dano ambiental - aqui pode ser uma empresa ou pessoa física.
        (devolva a string "Não encontrado" caso não exista)
        Elemento 5 da lista: Categoria do elemento 4 - "Pessoa Física" ou "Pessoa Jurídica"
        (devolva a string "Não encontrado" caso não exista)
        Elemento 6 da lista: A data de ocorrência do impacto ambiental no formato DD/MM/AA
        (devolva a string "Não encontrado" caso não exista)
        Elemento 7 da lista: A extensão da área afetada pelo impacto ambiental
        (devolva a string "Não encontrado" caso não exista)
        Elemento 8 da lista: A unidade de medida do elemento 7
        (devolva a string "Não encontrado" caso não exista)
        Elemento 9 da lista: A metodologia de punição do responsável pelo impacto
        (devolva a string "Não encontrado" caso não exista)
        Elemento 10 da lista: O valor da multa que o responsável deve pagar pelas suas ações (em R$)
        (devolva a string "Não encontrado" caso não exista)
        Texto bruto: {texto_extraido}"""
generation_config_low_temp = genai.GenerationConfig(
    max_output_tokens=500,
    temperature=0.0,
)
model = genai.GenerativeModel(model_name="gemini-1.5-flash")
resposta = model.generate_content(prompt, generation_config=generation_config_low_temp)
time.sleep(3)
resposta.text

"```\n[True, 'Não encontrado', 'PA', 'Novo Repartimento', 'INDUSTRIA E COMERCIO DE MADEIRAS JUREMA LTDA', 'Pessoa Jurídica', 'Não encontrado', '20,537 m³', 'm³', 'Multa diária e multa administrativa', 'R$ 6.300,00']\n```\n"

In [48]:
filtrados.iloc[1499,3]

'http://jud-anexos.digesto.com.br/4df0ebd528fe02fcb3c1868b80675c12.pdf'

In [49]:
print(texto_extraido)

 
 
 
 
PROCESSO JUDICIAL ELETRÔNICO
 
 
TRIBUNAL DE JUSTIÇA DO ESTADO DO PARÁ
 
 
VARA ÚNICA DA COMARCA DE NOVO REPARTIMENTO/PA
 
 
  
 
  
 
  
 
PROCESSO: 0800665-06.2021.8.14.0123
 
 
AUTOR: MINISTÉRIO PÚBLICO DO ESTADO DO PARÁ. 
 
REQUERIDO: INDUSTRIA E COMERCIO DE MADEIRAS JUREMA LTDA, pessoa jurídica,
CNPJ 04.680.044/0001-60, situada na estrada do Tuerê, s/n, Distrito de Maracajá, Zona Rural de
Novo Repartimento/PA.
 
 
 
DECISÃO/MANDADO 
 
 
 
1. Trata-se de AÇÃO CIVIL PÚBLICA com pedido de tutela de urgência proposta pelo Ministério
Público do Estado do Pará contra INDUSTRIA E COMERCIO DE MADEIRAS JUREMA LTDA,
devidamente qualificada nos autos, visando a obtenção de provimento antecipado determinando
a não exploração de recurso madeireiro, quer seja na modalidade de adquirir, receber, expor a
venda, guardar, ter em depósito e transportar madeira, lenha, carvão ou produto de origem
vegetal sem a autorização/licença para tanto sob pena de multa diária de R$30.000,00 (trinta mil
